In [28]:
# Importing the required libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
import cv2
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import argparse
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from torchvision.utils import save_image
from sklearn.model_selection import train_test_split

In [29]:
# constructing the argument parser
parser = argparse.ArgumentParser()

parser.add_argument('-e', '--epochs', type=int, default=40, 
            help='number of epochs to train the model for')

args = vars(parser.parse_args())

def save_decoded_image(img, name):
    img = img.view(img.size(0), 3, 224, 224)
    save_image(img, name)



usage: ipykernel_launcher.py [-h] [-e EPOCHS]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/hiteshsaaimanancherypanneerselvam/Library/Jupyter/runtime/kernel-75148bfc-3521-40a4-b1a9-a8483cb073c4.json


SystemExit: 2

In [8]:
# helper functions
image_dir = '../outputs/saved_images'

os.makedirs(image_dir, exist_ok=True)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(device)
batch_size = 2

cpu


In [17]:
# Accumulating the images for train and test

gauss_blur = os.listdir('input_images/gaussian_blurred')

# sorting the folder to align with sharp images folder
gauss_blur.sort()

sharp = os.listdir('input_images/sharp')

#sorting the folder to align with gaussian_blurred images folder
sharp.sort()

x_blur = []
for i in range(len(gauss_blur)):
    x_blur.append(gauss_blur[i])

y_sharp = []
for i in range(len(sharp)):
    y_sharp.append(sharp[i])



In [18]:
# Train Test Split

x_train, x_val, y_train, y_val = train_test_split(x_blur, y_sharp, test_size=0.20)

print(f"Train data instances: {len(x_train)}")
print(f"Validation data instances: {len(x_val)}")

# define transforms
transform = transforms.Compose([
    transforms.ToPILImage(), #  Transforming the image to PIL,
    transforms.Resize((224, 224)), # Transoforing the image to 224X224 dimension,
    transforms.ToTensor(), #Transforming the image to torch tensor,
])

class DeblurDataset(Dataset):
    def __init__(self, blur_paths, sharp_paths=None, transforms=None):
        self.X = blur_paths
        self.y = sharp_paths
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        blur_image = cv2.imread(f"../input/gaussian_blurred/{self.X[i]}")
        
        if self.transforms:
            blur_image = self.transforms(blur_image)
            
        if self.y is not None:
            sharp_image = cv2.imread(f"../input/sharp/{self.y[i]}")
            sharp_image = self.transforms(sharp_image)
            return (blur_image, sharp_image)
        else:
            return blur_image

train_data = DeblurDataset(x_train, y_train, transform)
val_data = DeblurDataset(x_val, y_val, transform)
 
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)


Train data instances: 280
Validation data instances: 70


In [30]:
class DeblurCNN(nn.Module):
    def __init__(self):
        super(DeblurCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=2)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        return x

model = DeblurCNN().to(device)
print(model)

DeblurCNN(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
)


In [22]:
# the loss function
criterion = nn.MSELoss()

# the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( 
        optimizer,
        mode='min',
        patience=5,
        factor=0.5,
        verbose=True
    )

def fit(model, dataloader, epoch):
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        blur_image = data[0]
        sharp_image = data[1]
        blur_image = blur_image.to(device)
        sharp_image = sharp_image.to(device)
        optimizer.zero_grad()
        outputs = model(blur_image)
        loss = criterion(outputs, sharp_image)
        # backpropagation
        loss.backward()
        # update the parameters
        optimizer.step()
        running_loss += loss.item()
    
    train_loss = running_loss/len(dataloader.dataset)
    print(f"Train Loss: {train_loss:.5f}")
    
    return train_loss

In [24]:
train_epoch_loss = fit(model, trainloader, epoch)

NameError: name 'epoch' is not defined

In [25]:
args['epochs']

NameError: name 'args' is not defined